# Machine Learning in Spark

Spark provides machine learning capabilities through the mllib library.  This notebook illustrates how simple regression and classification tasks can be performed in Spark using mllib.

The intent of the following exercises is to establish familiarity with Spark ML syntax and programming concepts.  In some cases we deviate from machine learning best practices in the interest of time.  We will take care to highlight such deviations in order to ensure that you do not come away from these exercises with an improper understanding of their real world use.

 ## 1.0 Regression with The Lasso

In this exercise we'll try to predict automobile fuel efficiency (e.g. MPG) based on vehicle characteristics like horsepower, cylinders, etc.  Each line in the data provides mileage for a single vehicle type.  Each row in the data is described by https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.names.  We will use this data to train a regression model for mileage.

Regression ML techniques predict a continuously valued output based on the value of one or more features.  The Lasso is a popular regression method that applies regularization to control variance and additionally perform feature selection.  Lasso regression is popular becuase it yields parsimonius models.  Information about the Lasso can be found at https://en.wikipedia.org/wiki/Least_squares#Lasso_method or http://statweb.stanford.edu/~tibs/lasso/simple.html  

We start by simply loading the file into a Spark RDD.  A row count and row sample are then printed for basic sanity checking.  Ensure that you see 398 rows and data that contains text like "chevrolet" or "plymouth".  Note that "sandbox.hortonworks.com" resolves to the IP address of the Hadoop name node in the lab VM.

In [247]:
raw = sc.textFile("hdfs://sandbox.hortonworks.com/user/spark/auto-mpg.data")
print raw.count()
print raw.take(5)

398
[u'18.0   8   307.0      130.0      3504.      12.0   70  1\t"chevrolet chevelle malibu"', u'15.0   8   350.0      165.0      3693.      11.5   70  1\t"buick skylark 320"', u'18.0   8   318.0      150.0      3436.      11.0   70  1\t"plymouth satellite"', u'16.0   8   304.0      150.0      3433.      12.0   70  1\t"amc rebel sst"', u'17.0   8   302.0      140.0      3449.      10.5   70  1\t"ford torino"']


Observe above that elements in each row are delimited by multiple spaces.  We thus tokenize this data by mapping a lambda function onto the RDD.  The lambda function uses Python's regex capabilities to split the string on each occurrence of more than one consecutive space.  

Because no data set is ever clean, we find that the auto data contains ? characters in the 4th column.  We thus get rid of the invalid data.  Of couse, in the real world data is precious and we'd likely use imputation techniques to populate the missing data rather than throwing the records away.

In [248]:
import re
data = raw.map(lambda x: re.split("\s{2,}",x))
print "Record count post split: %d" % (data.count())
data = data.filter(lambda x: x[3] != '?')
print "Record count post removal of ?: %d" % (data.count())

Record count post split: 398
Record count post removal of ?: 392


The following code simply converts all of the tokenized strings above to floating point values.  Two Spark RDDs are created.  The labels RDD contains the vehicle MPG data.  Spark uses the term label to denote the quantity to be predicted.  The features RDD stores vehicle features (e.g. cylinders, horesepower).

In [249]:
typed_data = data.map(lambda x: (float(x[0]), [float (y) for y in x[1:7]]))

labels = typed_data.map(lambda (label, features): label)
features = typed_data.map(lambda (label, features): features)


The Lasso performs better with scaled (mean zero, unit variance) features, so we use Spark's feature scaling tools to scale the data prior to model building.  Concretely, the code below subtracts the feature mean from each feature, and further divides each feature by its standard deviation.  The RDD zip method simply performs a pairwise combination of two RDDs.  Here we combine the labels and scaled_features RDDs into a RDD where each member is a tuple of the form (label, features).

In [251]:
from pyspark.mllib.feature import StandardScaler

scaler = StandardScaler().fit(features)

scaled_data = labels.zip(scaler.transform(features))

Spark regressors and classifiers work on LabeledPoint objects.  We thus map our features and labels to LabeledPoints.  We then train the model with some pre-selected hyper parameters (iterations, scale).

NOTE:  Values for the iterations and step parameters have been selected based on experimentation.  In a real analytics project model hyper parameters would be chosen by cross validation in order to improve model accuracy and ability to generalize.

In [252]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LassoWithSGD
from numpy import array

points = scaled_data.map(lambda (label, features): LabeledPoint(label, features))

model = LassoWithSGD.train(points, iterations=500, step=0.01)

print model



(weights=[-1.08736515851,-1.17717279824,-1.05152524284,-1.29783961669,0.589816900998,1.61062519545], intercept=0.0)


We now use the model to predict for each of the training data points.  A few predictions are printed out so you will have a sense of how the model performs on the training data.

In [253]:
import math

predictions = points.map(lambda p: (p.label, model.predict(p.features)))

print "Predictions Sample:"
samp = predictions.take(10)
for pred in samp:
    print "Value: %f  Prediction: %f  Difference: %f" % (pred[0], pred[1], math.fabs(pred[0]-pred[1]))



Predictions Sample:
Value: 18.000000  Prediction: 15.712755  Difference: 2.287245
Value: 15.000000  Prediction: 13.877207  Difference: 1.122793
Value: 18.000000  Prediction: 14.932751  Difference: 3.067249
Value: 16.000000  Prediction: 15.308615  Difference: 0.691385
Value: 17.000000  Prediction: 15.259168  Difference: 1.740832
Value: 15.000000  Prediction: 10.776205  Difference: 4.223795
Value: 14.000000  Prediction: 9.660309  Difference: 4.339691
Value: 14.000000  Prediction: 9.911672  Difference: 4.088328
Value: 14.000000  Prediction: 9.617773  Difference: 4.382227
Value: 15.000000  Prediction: 11.863013  Difference: 3.136987


Mean Squared Error (MSE) and Root MSE (RMSE) are common means of assessing regression performance.  Below we see that our Lasso model yields a RMSE of ~4.0.     A simple Lasso model trained in R yields a RMSE of ~3.4.

NOTE:  The reported measures of accuracy only assess performance of the model on the training data.  The measures are optimistically biased on the training data.  Intuitively, we expect the model to perform well on the data used to train it.  More realistic performance assessments are yielded via cross validation or hold out sets.  The provided measures are only intended as a sanity check to ensure that the exercise produces a model that roughly approximates what can be done in a commonly used tool like R.

In [255]:
MSE = predictions.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / predictions.count()
print("Mean Squared Error = " + str(MSE))
print("Root Mean Squared Error = " + str(math.sqrt(MSE)))

Mean Squared Error = 16.1418174529
Root Mean Squared Error = 4.01768807313


## 2.0 Classification with Linear SVM

In this exercise we'll try to predict the species of Iris flower based on sepal and petal measurements.  The Iris data set is commonly used as a benchmark for classification problems in statistics and ML.  Please see https://en.wikipedia.org/wiki/Iris_flower_data_set

Classification ML techniques attempt to predict a discrete output.  Examples include male/female, likely to buy/unlikely to buy, etc.  Spark has a variety of classification algorithms.  In this example we'll be using a Linear Support Vector Machine classifier.  A full explanation of the Linear SVM is found at https://en.wikipedia.org/wiki/Support_vector_machine#Linear_SVM.  Intuitively, Linear SVM tries to find a hyperplane that separates the data points with what is called maximum margin.  Maximum margin simply means we want the hyperplane to maximize the distance between points of different classes that are close together.

Again, the first thing we do is load the iris data that you uploaded from HDFS into a RDD called iris_raw.  A count and sample of records is then output.

In [260]:
iris_raw = sc.textFile("hdfs://sandbox.hortonworks.com/user/spark/iris.data")
print iris_raw.count()
print iris_raw.take(5)

151
[u'5.1,3.5,1.4,0.2,Iris-setosa', u'4.9,3.0,1.4,0.2,Iris-setosa', u'4.7,3.2,1.3,0.2,Iris-setosa', u'4.6,3.1,1.5,0.2,Iris-setosa', u'5.0,3.6,1.4,0.2,Iris-setosa']


The Linear SVM implementation provided with Spark 1.3.1 (the version we're  using) requires class labels to be 1.0 and 0.0 for positive and negative cases respectively.  The SVM implementation also does not support multi-class classification as of version 1.3.1.  Thus the following function maps labels to 1.0 or 0.0.  The positive class is configurable.

In [261]:
def map_iris_label(label, positive):
    if label == positive:
        return 1.0
    else:
        return 0.0

We're going to try to accurately identify examples of Iris Virginica.  The following code creates label and feature RDDs where the raw string data is converted to floating point data.

In [262]:
pos_class = 'Iris-virginica'

iris_data = iris_raw.map(lambda x: x.split(','))
iris_data = iris_data.filter(lambda x: len(x) > 1)
iris_features = iris_data.map(lambda x: [float(y) for y in x[0:4]])
iris_labels = iris_data.map(lambda x: map_iris_label(x[4], pos_class))

Again, the Linear SVM implementation in Spark requires scaled features so we scale them here.

In [263]:
iris_scaler = StandardScaler().fit(iris_features)
iris_scaled_data = iris_labels.zip(iris_scaler.transform(iris_features))

To train the model, we convert the raw floating point features to Spark LabeledPoints and train the classifer with default parameters.  

NOTE:  In a real analytics scenario we would of course optimize parameter selection by cross validation.  Some experiments have been conducted to identify parameters that balance performance and speed.

In [264]:
from pyspark.mllib.classification import SVMWithSGD
iris_pts = iris_scaled_data.map(lambda (v,f): LabeledPoint(v,f))

iris_model = SVMWithSGD.train(iris_pts, iterations=200, step=0.75)

Finally, we make use the model to predict the labels of the training data.  Notice that we can predict Iris Virginica with around ~96% accuracy.  A quick training of a linear SVM model in R on the same data yields 96-98% accuracy.

NOTE:  Again, the measures are only provided for illustrative purposes and should not be used to infer model performance on unseen data.

In [265]:
iris_preds = iris_pts.map(lambda p: (p.label, iris_model.predict(p.features)))

training_err = iris_preds.filter(lambda (v, p): v != p).count() / float(iris_pts.count())

print "Training Accuracy: %f" % (1.0-training_err)
print "Training Error: %f" % (training_err)



Training Accuracy: 0.960000
Training Error: 0.040000
